Source practice: https://www.youtube.com/watch?v=Jy4wM2X21u0&list=PLhhyoLH6IjfxeoooqP9rhU3HJIAVAJ3Vz&index=3

##Imports

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

##Load data

In [18]:
train_dataset = datasets.MNIST(download = True, root = 'dataset/', train = True, transform = transforms.ToTensor())

In [19]:
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)

In [20]:
test_dataset = datasets.MNIST(download = True, root = 'dataset/', train = False, transform = transforms.ToTensor())

In [21]:
test_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)

##Set device

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Hyperparameters

In [23]:
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

# Bidirectional LSTM

In [24]:
class BRNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(BRNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(
        input_size, hidden_size, num_layers, batch_first = True, bidirectional = True
    )
    self.fc = nn.Linear(hidden_size * 2, num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)

    out, _ = self.lstm(x, (h0, c0))
    out = self.fc(out[:, -1, :])

    return out

##Initialize network

In [25]:
model = BRNN(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, num_classes = num_classes).to(device)

##Loss and Optimizer

In [26]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

## Train Network

In [16]:
for epoch in range(num_epochs):
  for batch_idx, (data, targets) in enumerate(train_loader):
    #Get data to cuda if possible
    data = data.to(device = device).squeeze(1) ####
    targets = targets.to(device = device)

    #Forward
    scores = model(data)
    loss = criterion(scores, targets)

    #Backward
    optimizer.zero_grad()
    loss.backward()

    #Gradient descent or adam step
    optimizer.step()


## Check accuracy on training and test

In [27]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on test data")
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device = device).squeeze(1)
      y = y.to(device = device)

      scores = model(x)
      _, predictions = scores.max(1) #maximum values
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

  model.train()
  return num_correct

In [28]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 5919 / 60000 with accuracy 9.87
Checking accuracy on training data
Got 5919 / 60000 with accuracy 9.87


tensor(5919)